# 9. tensorflow exercise #
first define computation graph with python and then more effectively execution with c++

## 9.1 install ##
already installed (path)

## 9.2 first practice ##


In [1]:
import tensorflow as tf

x = tf.Variable(3, name = 'x')
y = tf.Variable(4, name = 'y')
f = x*x*y + y
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)
    
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

40


## 9.3 computation graph management ##

In [2]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [3]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


In [4]:
print(x1.graph is tf.get_default_graph())
tf.reset_default_graph()
print(x1.graph is tf.get_default_graph())

True
False


## 9.4 node value management ##

In [5]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
# example 1
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [6]:
# example 2
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


at example1, y, z is independ so session twice execute x,w <br>
at example2, y, z excute to together and so execute x, w one time.

## 9.5 linear-regression with tensorflow ##


In [7]:
import numpy as np
from sklearn.datasets import fetch_california_housing
import matplotlib.pyplot as plt

housing = fetch_california_housing()

def linear_regression(housing):
    reset_graph()
    m, n = housing.data.shape
    housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

    X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = 'x')
    y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = 'y')
    XT = tf.transpose(X)
    theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
    with tf.Session() as sess:
        theta_value = theta.eval()
        print(theta_value)

**normal equation**<br>
$ \hat\theta = (((X^T \cdot X)^{-1}) \cdot X^T) \cdot y) $

In [8]:
linear_regression(housing)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


## 9.6 gradient descent ##

### 9.6.1 direct compute gradient ###

In [9]:
from sklearn.preprocessing import StandardScaler

def direct_gradient(housing):
    reset_graph()
    m, n = housing.data.shape
    housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
    n_epochs = 1000
    learning_rate = 0.01
    scaled_housing_data_plus_bias = StandardScaler().fit_transform(housing_data_plus_bias)
    X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = 'X')
    y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = 'y')
    # random_unifrom : random tensor
    theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = 'theta')
    y_pred = tf.matmul(X, theta, name = 'predictions')

    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name = 'mse')
    gradients = 2/m * tf.matmul(tf.transpose(X), error)
    train_op = tf.assign(theta, theta - learning_rate*gradients)

    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(n_epochs+1):
            if epoch % 100 == 0:
                print('epoch ', epoch, ' MSE : ', mse.eval())
            sess.run(train_op)
        best_theta = theta.eval()
        #print(best_theta)


In [10]:
direct_gradient(housing)

epoch  0  MSE :  7.589867
epoch  100  MSE :  4.8741293
epoch  200  MSE :  4.8182063
epoch  300  MSE :  4.812053
epoch  400  MSE :  4.809471
epoch  500  MSE :  4.8077292
epoch  600  MSE :  4.806481
epoch  700  MSE :  4.805583
epoch  800  MSE :  4.8049335
epoch  900  MSE :  4.8044662
epoch  1000  MSE :  4.804129


### 9.6.2 differential ###

In [11]:
def use_diff(housing):
    reset_graph()
    m, n = housing.data.shape
    housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
    n_epochs = 1000
    learning_rate = 0.01
    scaled_housing_data_plus_bias = StandardScaler().fit_transform(housing_data_plus_bias)
    X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = 'X')
    y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = 'y')
    # random_unifrom : random tensor
    theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = 'theta')
    y_pred = tf.matmul(X, theta, name = 'predictions')

    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name = 'mse')
    
    gradients = tf.gradients(mse, [theta])[0]
    train_op = tf.assign(theta, theta - learning_rate*gradients)

    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(n_epochs+1):
            if epoch % 100 == 0:
                print('epoch ', epoch, ' MSE : ', mse.eval())
            sess.run(train_op)
        best_theta = theta.eval()

In [12]:
use_diff(housing)

epoch  0  MSE :  7.589867
epoch  100  MSE :  4.8741293
epoch  200  MSE :  4.818206
epoch  300  MSE :  4.812053
epoch  400  MSE :  4.80947
epoch  500  MSE :  4.8077297
epoch  600  MSE :  4.806481
epoch  700  MSE :  4.805583
epoch  800  MSE :  4.804933
epoch  900  MSE :  4.8044662
epoch  1000  MSE :  4.804129


### 9.6.3 optimizer ###


In [13]:
def use_optimizer(housing):
    reset_graph()
    m, n = housing.data.shape
    housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
    n_epochs = 1000
    learning_rate = 0.01
    scaled_housing_data_plus_bias = StandardScaler().fit_transform(housing_data_plus_bias)
    X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = 'X')
    y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = 'y')
    # random_unifrom : random tensor
    theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = 'theta')
    y_pred = tf.matmul(X, theta, name = 'predictions')

    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name = 'mse')
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
    train_op = optimizer.minimize(mse)

    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(n_epochs+1):
            if epoch % 100 == 0:
                print('epoch ', epoch, ' MSE : ', mse.eval())
            sess.run(train_op)
        best_theta = theta.eval()

In [14]:
use_optimizer(housing)

epoch  0  MSE :  7.589867
epoch  100  MSE :  4.8741293
epoch  200  MSE :  4.818206
epoch  300  MSE :  4.812053
epoch  400  MSE :  4.80947
epoch  500  MSE :  4.8077297
epoch  600  MSE :  4.806481
epoch  700  MSE :  4.805583
epoch  800  MSE :  4.804933
epoch  900  MSE :  4.8044662
epoch  1000  MSE :  4.804129


### 9.7 input data to training algorithm ###


In [15]:
A = tf.placeholder(tf.float32, shape=(None, 3))
a = [[[1, 2, 3]], [[4, 5, 6], [7, 8, 9]]]
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict = {A : a[0]})
    B_val_2 = B.eval(feed_dict = {A : a[1]})
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [16]:
def mini_batch_learning(housing):
    reset_graph()
    m, n = housing.data.shape
    housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
    scaled_housing = StandardScaler().fit_transform(housing_data_plus_bias)
    housing_target = housing.target.reshape(-1,1)
    
    epochs = 100
    X = tf.placeholder(tf.float32, shape = (None, n + 1), name = 'X')
    y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')
    
    batch_size = 100
    n_batches = int(np.ceil(m/batch_size))
    
    theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = 'theta')
    y_pred = tf.matmul(X, theta, name = 'predictions')
    error = y_pred - y
    
    optimizer = tf.train.MomentumOptimizer(learning_rate = 1e-3, momentum = 0.9)
    loss = tf.reduce_mean(tf.square(error), name = 'mse')
    train_op = optimizer.minimize(loss)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(epochs + 1):
            for batch_index in range(n_batches):
                X_batch, y_batch = batch_fetch(scaled_housing, housing_target, batch_index, batch_size)
                _, losses = sess.run([train_op, loss], feed_dict = {X : X_batch, y : y_batch})
            print('epoch : %i loss %f'%(epoch, losses), end = '\r')
    
def batch_fetch(data, target, batch_index, batch_size):
    X_batch = data[batch_index * batch_size: (batch_index+1)*batch_size]
    y_batch = target[batch_index * batch_size: (batch_index+1)*batch_size]
    
    return X_batch, y_batch
    

In [17]:
mini_batch_learning(housing)

## 9.8 model store & restore ##


In [20]:
def mini_batch_learning_with_saver(housing):
    reset_graph()
    m, n = housing.data.shape
    housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
    scaled_housing = StandardScaler().fit_transform(housing_data_plus_bias)
    housing_target = housing.target.reshape(-1,1)
    
    epochs = 100
    X = tf.placeholder(tf.float32, shape = (None, n + 1), name = 'X')
    y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')
    
    batch_size = 100
    n_batches = int(np.ceil(m/batch_size))
    
    theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = 'theta')
    y_pred = tf.matmul(X, theta, name = 'predictions')
    error = y_pred - y
    
    optimizer = tf.train.MomentumOptimizer(learning_rate = 1e-3, momentum = 0.9)
    loss = tf.reduce_mean(tf.square(error), name = 'mse')
    train_op = optimizer.minimize(loss)
    
    # store model
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(epochs + 1):
            for batch_index in range(n_batches):
                X_batch, y_batch = batch_fetch(scaled_housing, housing_target, batch_index, batch_size)
                _, losses = sess.run([train_op, loss], feed_dict = {X : X_batch, y : y_batch})
            if epoch % 10 == 0:
                save_path = saver.save(sess, './model_test.ckpt')
            print('epoch : %i loss %f'%(epoch, losses), end = '\r')

In [21]:
mini_batch_learning_with_saver(housing)

**model restore**<br>
with tf.Session() as sess: <br>
&nbsp;saver.restore(sess, 'path') <br>
<br>
another way.
<br>saver = tf.train.Saver({'weights' : theta})<br>
graph<br>
saver = tf.train.import_meta_graph('path/filename.ckpt.meta') <br>

## 9.9 visualize ##

In [34]:
from datetime import datetime
def mini_batch_learning_visualize(housing):
    reset_graph()
    m, n = housing.data.shape
    housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
    scaled_housing = StandardScaler().fit_transform(housing_data_plus_bias)
    housing_target = housing.target.reshape(-1,1)
    
    epochs = 100
    X = tf.placeholder(tf.float32, shape = (None, n + 1), name = 'X')
    y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')
    
    batch_size = 100
    n_batches = int(np.ceil(m/batch_size))
    
    theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = 'theta')
    y_pred = tf.matmul(X, theta, name = 'predictions')
    error = y_pred - y
    
    optimizer = tf.train.MomentumOptimizer(learning_rate = 1e-3, momentum = 0.9)
    loss = tf.reduce_mean(tf.square(error), name = 'mse')
    train_op = optimizer.minimize(loss)
    
    # store model
    saver = tf.train.Saver()
    # graph setting
    #now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
    _log = 'tf_logs/'
    mse_summary = tf.summary.scalar('MSE', loss)
    summary_op  = tf.summary.merge_all()
    
    with tf.Session() as sess:
        file_writer = tf.summary.FileWriter(_log, sess.graph)
        file_writer.flush()
        sess.run(tf.global_variables_initializer())
        for epoch in range(epochs + 1):
            for batch_index in range(n_batches):
                X_batch, y_batch = batch_fetch(scaled_housing, housing_target, batch_index, batch_size)
                _, losses = sess.run([train_op, loss], feed_dict = {X : X_batch, y : y_batch})
            if epoch % 10 == 0:
                summary_str = sess.run(summary_op, feed_dict = {X : X_batch, y : y_batch})
                step = epoch * n_batches + batch_index
                saver.save(sess, _log + 'model_visualize.ckpt', step)
                file_writer.add_summary(summary_str, step)
            print('epoch : %i loss %f'%(epoch, losses), end = '\r')
        print('')
        
    file_writer.close()

In [35]:
mini_batch_learning_visualize(housing)

epoch : 100 loss 0.277746


tensorboard --logdir 'dir'

In [118]:
print(housing.data.shape)

(20640, 8)


# tensorboard not working ~0327 #